In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict
import json
from threading import Thread
from tqdm import tqdm
from time import time

In [ ]:
def getHTML(url, headers=None):
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        html = r.content
        try:
            return bs(html, 'html').text
        except:
            try:
                return bs(html, 'html.parser').text
            except:
                return bs(html, 'lxml').text


def findMatch(pattern, text):
    match = re.search(pattern, text)
    if match:
        res = match.group().strip()
        if res:
            return res
        

def 构建成语及俗语(url):
    html = getHTML(url)
    if html:
        word = findMatch(r".+(?=的成语解释及意思)", html)
        if word:
            成语及俗语[word]['拼音'] = findMatch(r"(?<=拼音：\n).+", html)
            成语及俗语[word]['简拼'] = findMatch(r"(?<=简拼：\n).+", html)
            
            近义词 = findMatch(r"(?<=近义词：\n).+", html)
            if 近义词:
                成语及俗语[word]['近义词'] = 近义词.split("、")
            else:
                成语及俗语[word]['近义词'] = []
            
            反义词 = findMatch(r"(?<=反义词：\n).+", html)
            if 反义词:
                成语及俗语[word]['反义词'] = 反义词.split("、")
            else:
                成语及俗语[word]['反义词'] = []
            
            成语及俗语[word]['用法'] = findMatch(r"(?<=用法：\n).+", html)
            成语及俗语[word]['解释'] = findMatch(r"(?<=解释：\n).+", html)
            成语及俗语[word]['出处'] = findMatch(r"(?<=出处：\n).+", html)
            成语及俗语[word]['例子'] = findMatch(r"(?<=例子：\n).+", html)
            成语及俗语[word]['谒后语'] = findMatch(r"(?<=谒后语：\n).+", html)
            成语及俗语[word]['谜语'] = findMatch(r"(?<=谜语：\n).+", html)
            成语及俗语[word]['成语故事'] = findMatch(r"(?<=成语故事：\n).+", html)
            成语及俗语[word]['链接'] = url


def main(start_idx=0, end_idx=31251, step=10):
    for i in tqdm(range(start_idx, end_idx, step)):
        threads = []
        for j in range(i, i + step if i + step <= end_idx else end_idx):
            url = "http://cy.5156edu.com/html4/{}.html".format(j)
            t = Thread(target=构建成语及俗语, args=(url,))
            t.start()
            threads.append(t)
        for t in threads:
            t.join()

In [ ]:
成语及俗语 = defaultdict(dict)

if __name__ == '__main__':
    s = time()
    main(step=30)
    e = time()
    print("Total time: " + str(e - s))

In [ ]:
with open('成语及俗语词典_后17440.json', 'w') as f:
    json.dump(成语及俗语, f)